In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
summer_data = pd.read_excel('summer-products-with-rating-and-performance-filtered.xlsx', engine="openpyxl")
summer_data.head()

,title_orig,price,retail_price,units_sold,uses_ad_boosts,rating,rating_count,rating_five_count,rating_four_count,rating_three_count,...,shipping_option_price,shipping_is_express,countries_shipped_to,inventory_total,has_urgency_banner,merchant_title,merchant_info_subtitle,merchant_rating_count,merchant_rating,merchant_has_profile_picture
0,2020 Summer Vintage Flamingo Print Pajamas Se...,16.00,14,100,0,3.76,54,26.0,8.0,10.0,...,4,0,34,50,1.0,zgrdejia,(568 notes),568,4.128521,0
1,Women's Casual Summer Sleeveless Sexy Mini Dress,8.00,22,20000,1,3.45,6135,2269.0,1027.0,1118.0,...,2,0,41,50,1.0,SaraHouse,"83 % avis positifs (17,752 notes)",17752,3.899673,0
2,2020 New Arrival Women Spring and Summer Beach...,8.00,43,100,0,3.57,14,5.0,4.0,2.0,...,3,0,36,50,1.0,hxt520,86 % avis positifs (295 notes),295,3.989831,0
3,Hot Summer Cool T Shirt for Women Fashion Tops...,8.00,8,5000,1,4.03,579,295.0,119.0,87.0,...,2,0,41,50,NaN,allenfan,"(23,832 notes)",23832,4.020435,0
4,Women Summer Shorts Lace Up Elastic Waistband ...,2.72,3,100,1,3.10,20,6.0,4.0,2.0,...,1,0,35,50,1.0,youngpeopleshop,"85 % avis positifs (14,482 notes)",14482,4.001588,0


In [5]:
tags_list = list(summer_data["tags"])
print(tags_list[0])

tags_tokens_list = []
tags_count = []
for tags in tags_list:
    tags_tokens = tags.split(',')
    # Remove empty spaces in each token
    tags_tokens = [token.strip() for token in tags_tokens]
    # Lowercasing
    tags_tokens = [token.lower() for token in tags_tokens]
    tags_tokens_list.append(tags_tokens)
    tags_count.append(len(tags_tokens))

print(tags_tokens_list[0])
print(tags_count[0])


# Adding the tag counts to the summer_data and save
summer_data["tags_count"] = tags_count
summer_data.to_excel('summer-products-with-rating-and-performance-filtered-python.xlsx', index=False)

Summer,Fashion,womenunderwearsuit,printedpajamasset,womencasualshort,Women's Fashion,flamingo,loungewearset,Casual,Shirt,casualsleepwear,Shorts,flamingotshirt,Elastic,Vintage,Tops,tshirtandshortsset,Women,Sleepwear,Print,womenpajamasset,womennightwear,Pajamas,womensleepwearset
['summer', 'fashion', 'womenunderwearsuit', 'printedpajamasset', 'womencasualshort', "women's fashion", 'flamingo', 'loungewearset', 'casual', 'shirt', 'casualsleepwear', 'shorts', 'flamingotshirt', 'elastic', 'vintage', 'tops', 'tshirtandshortsset', 'women', 'sleepwear', 'print', 'womenpajamasset', 'womennightwear', 'pajamas', 'womensleepwearset']
24


PermissionError: [Errno 13] Permission denied: 'summer-products-with-rating-and-performance-filtered-python.xlsx'

In [ ]:
tags_df = pd.DataFrame()

# Create a list of all unique tags

all_tags = []
for tags in tags_tokens_list:
    all_tags.extend(tags)

unique_tags = list(set(all_tags))

tags_df["tag_name"] = unique_tags

# Count the number of times each tag appears in the all_tags list
num_products_where_tag_appears = []
for tag in unique_tags:
    count = 0
    for tags in tags_tokens_list:
        if tag in tags:
            count += 1
    num_products_where_tag_appears.append(count)

tags_df["num_products_where_tag_appears"] = num_products_where_tag_appears

# sort by the number of products where the tag appears
tags_df = tags_df.sort_values(by="num_products_where_tag_appears", ascending=False)
print(tags_df)

# Save the tags_df to a new excel file
tags_df.to_excel('summer-products-tags-analysis.xlsx', index=False)


             tag_name  num_products_where_tag_appears
2503  women's fashion                            1120
898            summer                            1106
254           fashion                             931
682             women                             826
836            casual                             753
...               ...                             ...
1097     highwaispant                               1
1096             tree                               1
1095          midriff                               1
1094     skinny jeans                               1
2619      fanscooling                               1

[2620 rows x 2 columns]


In [ ]:
# Now back to summer data

# We add two columns for each product: the popular tag count and the unpopular tag count
# The threshold of popularity is 10 counts from num_products_where_tag_appears column

popular_tag_count = []
unpopular_tag_count = []

for tags in tags_tokens_list:
    popular_count = 0
    unpopular_count = 0
    for tag in tags:
        if tags_df[tags_df["tag_name"] == tag]["num_products_where_tag_appears"].values[0] >= 10:
            popular_count += 1
        else:
            unpopular_count += 1
    popular_tag_count.append(popular_count)
    unpopular_tag_count.append(unpopular_count)

# As a whole, popular_tag_counts and unpopular_tag_counts sum should be equal to tags_count, pairwise

for i in range(len(tags_count)):
    assert popular_tag_count[i] + unpopular_tag_count[i] == tags_count[i]

# Saving the popular and unpopular tag counts to the summer_data
    
summer_data["popular_tag_count"] = popular_tag_count
summer_data["unpopular_tag_count"] = unpopular_tag_count
summer_data.to_excel('summer-products-with-rating-and-performance-filtered-python.xlsx', index=False)

In [7]:
# Merchant info subtitle processing

merchant_info_subtitle_list = list(summer_data["merchant_info_subtitle"])

print(merchant_info_subtitle_list[0])

# In each line there is also a percent of positive review, which follows by percent mark

merchant_positive_review_percent = []

for subtitle in merchant_info_subtitle_list:
    if "%" in str(subtitle):
        # Check if the percent value before it is attached to the percent sign or not
        subtitle_temp = subtitle.replace("%", " %")
        # percent could be anywhere in the subtitle
        subtitle_tokens = subtitle_temp.split()
        # find the token that preceeds the token '%'
        percent_index = subtitle_tokens.index('%') - 1
        percent = subtitle_tokens[percent_index]
        percent = int(percent)
        merchant_positive_review_percent.append(percent)
    else: 
        merchant_positive_review_percent.append(None)

print(merchant_positive_review_percent)

# Add the merchant positive review percent to the summer_data and save

summer_data["merchant_positive_review_percent"] = merchant_positive_review_percent

# delete the column merchant_info_subtitle

# copy the summer_data to a new dataframe
summer_data_copy = summer_data.copy()
del summer_data_copy["merchant_info_subtitle"]
summer_data_copy.to_excel('summer-products-with-rating-and-performance-filtered-python.xlsx', index=False)

(568 notes)
[None, 83, 86, None, 85, 75, 86, None, 82, 85, 84, 84, 87, 83, None, 86, None, 84, 87, 86, 91, None, 90, None, 85, 87, 90, None, 81, 86, 84, 87, 81, 92, None, None, 85, 85, 83, 85, 88, None, 87, 93, 89, 86, 81, 87, None, None, 89, 88, 81, 85, 87, None, 81, 85, 79, 90, 94, 88, None, None, 84, 88, 88, 88, None, 82, 88, 82, 91, 88, 87, 81, 91, 84, 89, 83, None, None, None, 89, 84, 87, 90, None, 84, 81, 86, 87, 83, 83, 86, 86, 90, 78, 80, None, None, None, None, 81, None, 83, 85, 82, 82, 86, 89, None, 84, 97, 85, 92, 82, 89, None, 85, 86, 83, 84, 75, 87, 73, None, 86, 86, 85, 86, None, None, None, 89, 92, 87, 91, 82, 87, None, None, None, 82, 88, 90, 88, 92, 82, 84, 77, 85, 83, 89, 83, 88, None, 85, 83, 85, 86, 86, 84, 93, 81, 84, None, 89, None, 89, 89, 84, 79, 84, 65, None, 87, None, None, None, 89, 81, 82, 88, 80, None, None, None, 86, 89, 88, 86, 88, 87, 92, 85, 86, 81, 88, 86, 84, None, None, 89, 82, None, 79, 80, 87, 91, 91, 85, None, 82, 91, 87, None, 88, 84, 90, 83, 90,